# Task 3 - Train on NEU train with the NSL after Histogram Equalization

In [1]:
import os
import numpy as np


In [2]:
train_crazing_dir = os.path.join('data/NEU-DET-HIS/train/images/crazing')
train_inclusion_dir = os.path.join('data/NEU-DET-HIS/train/images/inclusion')
train_patches_dir = os.path.join('data/NEU-DET-HIS/train/images/patches')
train_pitted_surface_dir = os.path.join('data/NEU-DET-HIS/train/images/pitted_surface')
train_rolledin_scale_dir = os.path.join('data/NEU-DET-HIS/train/images/rolled-in_scale')
train_scratches_dir = os.path.join('data/NEU-DET-HIS/train/images/scratches')

In [3]:
train_crazing_names = os.listdir(train_crazing_dir)
train_inclusion_names = os.listdir(train_inclusion_dir)
train_patches_names = os.listdir(train_patches_dir)
train_pitted_surface_names = os.listdir(train_pitted_surface_dir)
train_rolledin_scale_names = os.listdir(train_rolledin_scale_dir)
train_scratches_names = os.listdir(train_scratches_dir)

In [4]:
print('total training crazing images:', len(train_crazing_names))
print('total training inclusion images:', len(train_inclusion_names))
print('total training patches images:', len(train_patches_names))
print('total training pitted_surface images:', len(train_pitted_surface_names))
print('total training rolled_in_scale images:', len(train_rolledin_scale_names))
print('total training scratches images:', len(train_scratches_names))

total training crazing images: 241
total training inclusion images: 241
total training patches images: 241
total training pitted_surface images: 241
total training rolled_in_scale images: 241
total training scratches images: 240


In [5]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)

In [6]:
model = tf.keras.models.Sequential([
                                    # First Convolution
                                    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
                                    tf.keras.layers.MaxPooling2D(2, 2),
                                    # Second Convolution
                                    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2, 2),
                                    # Third Convolution
                                    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2, 2),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(512, activation='relu'),
                                    tf.keras.layers.Dense(512, activation='relu'),
                                    tf.keras.layers.Dense(6, activation='softmax'),])




In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 32)      9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 32)        9248      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 26, 26, 32)       0

In [8]:
import neural_structured_learning as nsl

In [9]:
# Wrap the model with adversarial regularization.
adv_config = nsl.configs.make_adv_reg_config(multiplier=0.2, adv_step_size=0.05)
adv_model = nsl.keras.AdversarialRegularization(model, adv_config=adv_config)

In [10]:
# Compile, train, and evaluate.
adv_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image

# Rescaling images
data_generator = ImageDataGenerator(rescale=1/255)

train_generator = data_generator.flow_from_directory(
    directory = 'data/NEU-DET-HIS/train/images/',
    target_size = (224, 224),
    batch_size = 60,
    class_mode = 'categorical',
    subset='training'
)

validation_generator = data_generator.flow_from_directory(
    directory = 'data/NEU-DET-HIS/validation/images/',
    target_size = (224, 224),
    batch_size = 60,
    class_mode = 'categorical'
)

Found 1440 images belonging to 6 classes.
Found 360 images belonging to 6 classes.


In [12]:
X_train, y_train = next(train_generator)
y_train = np.argmax(y_train, axis=1)
X_test, y_test = next(validation_generator)
y_test = np.argmax(y_test, axis=1)

In [13]:
history = adv_model.fit({'feature': X_train, 'label': y_train}, batch_size=32, epochs=5)

Epoch 1/5


C:\Users\91800\anaconda3\envs\MTP\lib\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


1/2 [==============>...............] - ETA: 4s - loss: 2.1484 - sparse_categorical_crossentropy: 1.7893 - sparse_categorical_accuracy: 0.1562 - scaled_adversarial_loss: 0.3591WARNING:tensorflow:The dtype of the watched tensor must be floating (e.g. tf.float32), got tf.int64


2/2 [==============================] - 7s 3s/step - loss: 2.9772 - sparse_categorical_crossentropy: 2.4798 - sparse_categorical_accuracy: 0.1333 - scaled_adversarial_loss: 0.4974
Epoch 2/5


1/2 [==============>...............] - ETA: 3s - loss: 2.2121 - sparse_categorical_crossentropy: 1.8424 - sparse_categorical_accuracy: 0.2500 - scaled_adversarial_loss: 0.3697WARNING:tensorflow:The dtype of the watched tensor must be floating (e.g. tf.float32), got tf.int64


...

2/2 [==============================] - 7s 3s/step - loss: 2.1737 - sparse_categorical_crossentropy: 1.8107 - sparse_categorical_accuracy: 0.2667 - scaled_adversarial_loss: 0.3630
Epoch 4/5


1/2 [==============>...............] - ETA: 3s - loss: 2.0676 - sparse_categorical_crossentropy: 1.7224 - sparse_categorical_accuracy: 0.4062 - scaled_adversarial_loss: 0.3452WARNING:tensorflow:The dtype of the source tensor must be floating (e.g. tf.float32) when calling GradientTape.gradient, got tf.int64


2/2 [==============================] - 7s 3s/step - loss: 2.0832 - sparse_categorical_crossentropy: 1.7353 - sparse_categorical_accuracy: 0.2667 - scaled_adversarial_loss: 0.3479
Epoch 5/5


1/2 [==============>...............] - ETA: 3s - loss: 2.0253 - sparse_categorical_crossentropy: 1.6870 - sparse_categorical_accuracy: 0.6562 - scaled_adversarial_loss: 0.3383WARNING:tensorflow:The dtype of the source tensor must be floating (e.g. tf.float32) when calling GradientTape.gradient, got tf.int64


2/2 [==============================] - 7s 3s/step - loss: 2.0485 - sparse_categorical_crossentropy: 1.7063 - sparse_categorical_accuracy: 0.3667 - scaled_adversarial_loss: 0.3422


In [14]:
history.history

{'loss': [2.977160930633545,
  2.311278820037842,
  2.173699378967285,
  2.08317232131958,
  2.0484752655029297],
 'sparse_categorical_crossentropy': [2.479759454727173,
  1.9251289367675781,
  1.810719609260559,
  1.7353184223175049,
  1.7063243389129639],
 'sparse_categorical_accuracy': [0.13333334028720856,
  0.15000000596046448,
  0.2666666805744171,
  0.2666666805744171,
  0.36666667461395264],
 'scaled_adversarial_loss': [0.49740147590637207,
  0.38614988327026367,
  0.3629799485206604,
  0.3478539288043976,
  0.3421509265899658]}

## Test on NEU test, and whole ENEU after Histogram Equalization


In [15]:
adv_model.evaluate({'feature': X_test, 'label': y_test})

1/2 [==============>...............] - ETA: 1s - loss: 2.1030 - sparse_categorical_crossentropy: 1.7515 - sparse_categorical_accuracy: 0.1250 - scaled_adversarial_loss: 0.3514WARNING:tensorflow:The dtype of the source tensor must be floating (e.g. tf.float32) when calling GradientTape.gradient, got tf.int64


2/2 [==============================] - 3s 2s/step - loss: 2.0915 - sparse_categorical_crossentropy: 1.7420 - sparse_categorical_accuracy: 0.2000 - scaled_adversarial_loss: 0.3495


[2.0914511680603027,
 1.741976261138916,
 0.20000000298023224,
 0.3494751751422882]

In [16]:
validation_generator_ENEU = data_generator.flow_from_directory(
    directory = 'images_preprocessed\images_histeq_resize\ENEU',
    target_size = (1,224, 224),
    batch_size = 60,
    class_mode = 'categorical'
)

Found 11160 images belonging to 7 classes.


In [17]:
X_test_ENEU, y_test_ENEU = next(validation_generator_ENEU)
X_test_ENEU = np.argmax(X_test_ENEU, axis=1)
y_test_ENEU = np.argmax(y_test_ENEU, axis=1)

In [18]:
adv_model.evaluate({'feature': X_test_ENEU, 'label': y_test_ENEU})

1/2 [==============>...............] - ETA: 1s - loss: 2.1620 - sparse_categorical_crossentropy: 1.8016 - sparse_categorical_accuracy: 0.1562 - scaled_adversarial_loss: 0.3603WARNING:tensorflow:The dtype of the source tensor must be floating (e.g. tf.float32) when calling GradientTape.gradient, got tf.int64


2/2 [==============================] - 3s 2s/step - loss: 2.1536 - sparse_categorical_crossentropy: 1.7946 - sparse_categorical_accuracy: 0.1833 - scaled_adversarial_loss: 0.3589


[2.1535797119140625,
 1.7946497201919556,
 0.18333333730697632,
 0.35892993211746216]